In [7]:
# ===============================
# PHASE 5 — RISK FUSION & DASHBOARD
# ===============================

import pandas as pd
import numpy as np

# Load predictions
xgb = pd.read_csv("../data/processed/xgb_predictions.csv")[['txId','fraud_prob']]
iso = pd.read_csv("../data/processed/if_predictions.csv")[['txId','anomaly_score']]
gnn = pd.read_csv("../data/processed/gnn_predictions.csv")[['txId','gnn_fraud_prob']]

# Merge all on txId
df = gnn.merge(xgb, on='txId', how='left').merge(iso, on='txId', how='left')

# Normalize anomaly score between [0,1]
df['anomaly_score_norm'] = (df['anomaly_score'] - df['anomaly_score'].min()) / \
                           (df['anomaly_score'].max() - df['anomaly_score'].min())

# Risk Fusion
df['risk'] = (0.50 * df['fraud_prob']) + \
             (0.30 * df['gnn_fraud_prob']) + \
             (0.20 * df['anomaly_score_norm'])

df['risk_score'] = (df['risk'] * 100).round(2)

# Alert mapping
def alert_level(score):
    if score >= 80: return "⚠️ CRITICAL — FLAG Wallet"
    elif score >= 60: return "🟡 High Risk — Add to Watchlist"
    elif score >= 40: return "🔵 Medium — Monitor"
    return "🟢 Low Risk"

df['alert'] = df['risk_score'].apply(alert_level)

# Sort by riskiest
top10 = df.sort_values(by='risk_score', ascending=False).head(10)

print("\n=== TOP 10 RISKIEST TRANSACTIONS ===")
print(top10[['txId','risk_score','fraud_prob','gnn_fraud_prob','anomaly_score_norm','alert']])

# Save dashboard results
df.to_csv("../data/processed/final_risk_scored.csv", index=False)
print("\nSaved: data/processed/final_risk_scored.csv")



=== TOP 10 RISKIEST TRANSACTIONS ===
            txId  risk_score  fraud_prob  gnn_fraud_prob  anomaly_score_norm  \
85622  115949445       83.94    0.993280        0.999396            0.214726   
93844  149161642       83.18    0.999594        0.987472            0.178984   
19992   29300677       83.08    0.999918        0.993080            0.164465   
25195   39687411       83.08    0.999954        0.993927            0.163054   
25188   39684200       82.91    0.999845        0.992923            0.156326   
94855  151227267       82.87    0.999765        0.987391            0.163200   
4348     6245395       82.75    0.997343        0.998051            0.146912   
21961   31168385       82.49    0.988920        0.973222            0.192242   
69436   96958874       82.25    0.999801        0.985132            0.135282   
92919  148569565       82.16    0.999832        0.979826            0.138561   

                           alert  
85622  ⚠️ CRITICAL — FLAG Wallet  
93844  ⚠️ C

In [5]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Extract embeddings from model logits
embeddings = out_all.detach().cpu().numpy()

# Reduce to 2D
pca = PCA(n_components=2)
emb_2d = pca.fit_transform(embeddings)

labels = full_data['binary_label'].values

plt.figure(figsize=(8,8))
plt.scatter(emb_2d[labels==0, 0],
            emb_2d[labels==0, 1],
            s=2, alpha=0.3, label='Normal', color='blue')

plt.scatter(emb_2d[labels==1, 0],
            emb_2d[labels==1, 1],
            s=4, alpha=0.9, label='Fraud', color='red')

plt.title("GNN Embedding Clusters: Fraud vs Normal")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()
plt.show()


NameError: name 'out_all' is not defined